In [1]:
# IR HW2 BM25L model
import numpy as np
from math import log
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import normalize
path = './ntust-ir-2020/'

In [2]:
# get doc files and query files
doc = []
doc_list = []
query = []
query_list = []
terms_set = set()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        doc.append(f_temp.read().split())
        f_temp.close()
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp = f_temp.read().split()
        query.append(temp)
        for w in temp: terms_set.add(w)
        f_temp.close()

In [3]:
# Calculate doc average length
doc_len = []
for d in doc:
    doc_len.append(len(d))
doc_avg = np.average(doc_len)
query_len = []
for q in query:
    query_len.append(len(q))
d_len = len(doc)
q_len = len(query)
terms_set = list(terms_set)
terms_len = len(terms_set)

In [4]:
sim_bm = np.zeros([q_len, d_len], dtype=float) # q_len * d_len

In [5]:
idf_dic = dict()
for t in terms_set:
    c = 0
    for d in range(d_len):
        if(t in doc[d]): c+=1
    idf_dic[t] = c

In [12]:
K1 = 3
K3 = d_len
b = 0.85
N = d_len
delta = 0.4

In [13]:
for q in tqdm(range(q_len)):
    for d in range(d_len):
        sum = 0
        for w in query[q]:
            idf_bm = 0
            w_query = 0
            w_doc = 0
            if(w in doc[d]):
                # idf
                ni = idf_dic[w]
                idf_bm = log((N - ni + 0.5) / (ni + 0.5))
                # w_query
                w_query = (K3 + 1) / (K3 + 1)
                # w_doc
                tf_d = doc[d].count(w)
                tf_d_p = tf_d / (1 - b + b * doc_len[d] / doc_avg)
                if(not (tf_d_p > 0)): tf_d_p = 0
                w_doc = (K1 + 1) * (tf_d_p + delta) / (K1 + tf_d_p + delta)
            sum += w_doc * w_query * idf_bm
        sim_bm[q][d] = sum

In [14]:
# Create output file
fp = open("vsm_result_k1_{}_b{}_s_{}.txt".format(K1, b, delta), "w")
fp.write("Query,RetrievedDocuments\n")
for q in tqdm(range(q_len)):
    fp.write('{},'.format(query_list[q]))
    for s in np.argsort(-sim_bm[q]):
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()